In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from paretoset import paretoset

from src.io import load_results

pd.options.plotting.backend = "plotly"
from src.scoring import get_details

In [2]:
result_df = load_results()
summary_df, eligible_points_df, score_df  = get_details(result_df)

                    time  score  eligible_points  percentage
baseline_random     0.03   3.72               12        0.31
saabas              0.06   5.04               10        0.50
tree_shap           0.04   6.06               10        0.61
sage               98.57   3.97                6        0.66
kernel_shap        21.51   9.13               10        0.91
lime             1417.11   8.56               12        0.71
maple             567.10   8.23               12        0.69


In [ ]:
min_x_axis_value = .01
def pareto(summary_df, title="Global performance of xAI methods"):
    summary_df.loc[summary_df.time < min_x_axis_value, 'time'] = min_x_axis_value
    if summary_df.percentage.max() < 1.:
        summary_df.percentage = summary_df.percentage * 100
    summary_df['explainer_name'] = summary_df.index
    # fig = go.Figure()
    fig = px.scatter(summary_df,
                     x='time',
                     y='percentage',
                     # s='explainer_name', # https://www.geeksforgeeks.org/how-to-annotate-matplotlib-scatter-plots/
                     # c='eligible_points',
                     text='explainer_name',
                     log_x=True,
                     labels={
                         "time": "Time (seconds)",
                         "percentage": "Percentage %",
                         'eligible_points': 'maximum score',
                         'explainer_name': 'Explainer'
                     },
                     title=title,
                     )

    mask = paretoset(summary_df[['percentage', 'time']], sense=["max", "min"])
    pareto_df = summary_df[mask].sort_values('time')
    fig.add_trace(go.Line(x=pareto_df.time,  #logx=True,
                          y=pareto_df.percentage,
                          text=pareto_df.explainer_name,
                          # textposition='middle right',
                          name='Pareto front',
                          ))
    fig.show();

pareto(summary_df)

In [ ]:
from src.scoring import restrict_tests

result_df_restricted = restrict_tests(result_df,
                                      criteria={'importance': True,
                                                'attribution': False,
                                                'interaction': False, },
                                      supported_model=None)
summary_df_restricted, eligible_points_df_restricted, score_df_restricted  = get_details(result_df_restricted)
pareto(summary_df_restricted, title="Performance of xAI methods: Feature Importance only")

In [ ]:
# todo choose options to have a more fair comparaison

# mention that sage is not comparable
# Pareto front

# test type: importance, attribution, interaction
# todo [after acceptance] gpu / cpu
# model agnostic / tree based / ...
